# Project File - APS360 Team 25
Divided into the following section: 
# 
1) Library imports
2) Data imports
3) Model architecture definition
4) Training function definition
5) Model training
6) Model testing

## Library imports 
(Place all library imports here)

In [3]:
#KP - I just added the main ones from the labs.
import torch
import numpy as np

#import torchvision
import matplotlib.pyplot as plt
import torch
import torch.nn as nn

import time # Tracking model training time.

## Data imports

In [4]:
#@Sergio

## Model architecture definition

Set the hyperparameters below:


In [5]:
#For LSTM: https://pytorch.org/docs/stable/generated/torch.nn.LSTM.html#torch.nn.LSTM 
n1 = 1 #Number of input features
nh1 = 1 #Number of features in the hidden state h
nl1 = 1 #Number of layers in each LSTM calling (more than 1 stacks them with same number of hidden features).
dropout1 = 0 #Percentage dropout of each layer. I believe 0-->1 range.

n2 = 1 #Number of input features
nh2 = 1 #Number of features in the hidden state h
nl2 = 1 #Number of layers in each LSTM calling (more than 1 stacks them with same number of hidden features).
dropout2 = 0 #Percentage dropout of each layer. I believe 0-->1 range.

In [7]:
torch.manual_seed(1000) #Set the manual seed so that we get reproducible results.

#Define the RNNMusicGenerator model below:
class RNNMusicGenerator(nn.Module): 
    def __init__(self):
        super(RNNMusicGenerator, self).__init__()
        self.name = "MusicGen"
        self.LSTM1 = nn.LSTM(input_size = n1, hidden_size = nh1,num_layers = nl1, batch_first = True,dropout = dropout1)
        #^^Expects data as: (batch_size,sequence,features)
        self.LSTM2 = nn.LSTM(input_size = n, hidden_size = nh,num_layers = nl, batch_first = True,dropout = dropout)
        #unsure of the output dimensions still...
    def forward(self, x):
        x = (F.relu(self.LSTM1(x)))
        x = self.LSTM2(x) #No activation function on second layer for now...
        return x
    
print('Model class created succesfully')

Model class created succesfully


## Training function

In [ ]:
#To help us save the model easier...
def get_model_name(name, batch_size, learning_rate, epoch):
    """ Generate a name for the model consisting of all the hyperparameter values

    Args:
        config: Configuration object containing the hyperparameters
    Returns:
        path: A string with the hyperparameter name and value concatenated
    """
    path = "model_{0}_bs{1}_lr{2}_epoch{3}".format(name,
                                                   batch_size,
                                                   learning_rate,
                                                   epoch)
    return path


In [8]:
def train(model,data, num_epochs=5, batch_size=64, learning_rate=1e-3):
    torch.manual_seed(1000) #Fixed. Make sure we use this throughout...
    criterion = nn.MSELoss() # mean square error loss. Compares reconstruction errors entry by entry.
    optimizer = torch.optim.Adam(model.parameters(),
                                 lr=learning_rate, 
                                 weight_decay=1e-5) # <-- Sometimes Adam converges faster than SGD
    train_loader = torch.utils.data.DataLoader(data, 
                                               batch_size=batch_size, 
                                               shuffle=True)
    
    iters, losses = [], []
    
    n = 0 # the number of iterations
    start_time=time.time() #Start of training.
    outputs = []
    
    for epoch in range(num_epochs):
        for data in train_loader:
            excerpt = data[:-1] # Extracts all but the last note (model will predict last note)
            true_note = data[-1] # Extracts the last note.
            pred_note = model(excerpt) # Collects next note prediction.
            loss = criterion(pred_note, true_note)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad() # Clean up, clean up, everybody, everywhere...

                
            iters.append(n)
            losses.append(float(loss)/batch_size)             # compute *average* loss
            
            n += 1
        
        print('Epoch:{}, Loss:{:.4f}'.format(epoch+1, float(loss)))
        outputs.append((epoch, true_note, pred_note),)
      #Checkpoint the model every epoch
        model_path = get_model_name(model.name, batch_size, learning_rate, epoch) #Returns the model name for 
      #the save file.
        print('model_path: ',model_path)
        torch.save(model.state_dict(), model_path) #Saves the current model with the weights.
        print('model checkpointed')
    end_time= time.time()
    
    
      # plotting
    plt.title("Training Curve")
    plt.plot(iters, losses, label="Train")
    plt.xlabel("Iterations")
    plt.ylabel("Loss")
    plt.show()

    return outputs

## Model Training

In [9]:
model = RNNMusicGenerator()

train(model,data,batch_size = 1,num_epochs = 1,lr = 0.0001)

NameError: name 'n' is not defined

## Model Testing

Since our model is 'tested' with people listening to it, we need to just generate some samples.

In [10]:
#Using random noise inputs:

x = torch.randn(1, 1, 1) # (Batch_size,sequence,notes) --> Must match the LSTM layer input.

song_length = 1 #How many new notes we want the model to generate.

start = True
for t in range(0,song_length): #Generates t new notes.
    new_sample = model(x)
    if start:
        new_excerpt = torch.cat(x,new_sample) #initializes new_excerpt as the random noise + new note
    else:
        new_excerpt = torch.cat(new_excerpt,new_sample)
    x = torch.cat(x[1:],new_sample) #Update x by throwing out the first note in the sequence and shoving the new note to the end.

print('loop done')

NameError: name 'model' is not defined